# R0 factor sensitivity for Robot Vaccine: SP

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime
import itertools
import matplotlib.pylab as plt
from matplotlib import rc
rc("text", usetex=True)
rc("font", family="serif")

import run_robot
import prepare_data
from importlib import reload
reload(prepare_data)
reload(run_robot)

In [ ]:
# Define the basic data for the case studies

# Mean incubation time
tinc = 5.2

# Mean inffectious time
tinf = 2.9

# Basic reproduction number
basic_rep = 2.5

# Lockdown level
lock_level = 1.0

# Simulation horizon
# A little more than a year when thevaccine should be here
ndays = 7*52

# Mean time in ICU
time_icu = 7

# Transmission/infection effect of the vaccine - Oxford profile
oxford_atten = [1.0, 0.24, 0.176]
# Transmission/infection effect of the vaccine - Pfizer profile
pfizer_atten = [1.0, 0.15, 0.05]

# R0 factor for each population
r0pop = [1.0, 1.3, 1.0, 1.0]

# Max dose profile
max_doses = 0.015*np.ones(ndays)
max_doses[0:100] = 0.0025
max_doses[100:150] = 0.005

# Doses window
doses_min_window, doses_max_window = [28], [84]

# Define basic paramters
basic_prm = prepare_data.save_basic_parameters(tinc=tinc, tinf=tinf, rep=basic_rep, min_level=lock_level,
    time_icu=time_icu, alternate=0, window=14, ndays=ndays)

pre_cities_data = {
    "S1": [0.685], 
    "E1": [0.01], 
    "I1": [0.006], 
    "R1": [0.299], 
    "population": [44639899], 
    "icu_capacity": [0.000175], 
    "start_date": [pd.to_datetime("2020-2-1")]
}
cities_data = pd.DataFrame(pre_cities_data, index=["SP"])
cities_data

In [ ]:
# Create a target matrix (max infected level)
ncities, ndays = len(cities_data.index), int(basic_prm["ndays"])
target = 0.8*np.ones((ncities, ndays))
target = prepare_data.save_target(cities_data, target)

# Use a forcedif that releases the cities in the end
force_dif = np.zeros((ncities, ndays))

In [ ]:
# Add information on the time series that estimate the need of ICUs
# We are using the time series adjusted considering that the mean ICU stay is 7 days (which lead to larger ICU capacity).

# Define the time series to estimate ICU demand
ts_sp = np.array([0.009, 0.015, 0.492, 0.0, 0.0, 0.0, 0.083, 0.011, 0.011])

ts_drs = ts_sp*np.ones((len(cities_data), len(ts_sp)))
ts_drs = pd.DataFrame(data=ts_drs, index=cities_data.index, columns=[
    "rho_min", "rho_max", "intercept", "trend", "phi_1", "phi_2", "sigma_omega", "state0", "state_less_1"
])
ts_drs["confidence"] = 0.9
ts_drs["time_icu"] = time_icu
cities_data = pd.concat([cities_data, ts_drs], axis=1)
cities_data

In [ ]:
pd.set_option("display.width", 120)

# Simple function to run a test and save results
def run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0factor, r_atten, icu_atten, 
    max_doses, doses_min_window, doses_max_window, verbosity=1):
    run_robot.prepare_optimization(basic_prm, cities_data, M, target, hammer_data, force_dif,
        r0pop, r_atten, icu_atten, max_doses, doses_min_window, doses_max_window, verbosity=verbosity)
    run_robot.optimize_and_show_results(basic_prm, figure_file, result_file, cities_data, target, verbosity=verbosity)

In [ ]:
# Define mobility matrix.
M = prepare_data.convert_mobility_matrix_and_save(cities_data, max_neighbors=5, drs="data/report_drs_mobility.csv")
hammer_data = prepare_data.save_hammer_data(cities_data, 0, basic_prm["min_level"])
run_robot.find_feasible_hammer(basic_prm, cities_data, M, target, hammer_data, out_file=None, 
    incr_all=True, verbosity=1)
hammer_data["duration"] = 14
M, hammer_data

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

def agregate(solution, labels):
    values = solution.loc[labels[0]].sum()
    for i in range(1, len(labels)):
        values += solution.loc[labels[i]].sum()
    return values

def retrieve_vaccination_profile(sol, pop=0, dose=0):
    sol = sol.loc[:, pop, :]
    seir = sol.loc["s", :] + sol.loc["e", :] + sol.loc["i", :] + sol.loc["r", :]
    applied = sol.loc["v", dose]*seir.loc[dose]
    return applied[14:]

## With vaccination blocking infections

In [ ]:
print("******************** Running vaccine simulation")
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]

r_atten = pfizer_atten
icu_atten = [1.0, 1.0, 1.0]

free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_infection_{max_dose_thresold}.csv"
    figure_file = f""
    run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, 
        max_doses, doses_min_window, doses_max_window)

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.8*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Relaxed days ($R0 \geq 2.0$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

In [ ]:
free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_infection_{max_dose_thresold}.csv"
    figure_file = f""

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] >= basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Fully open ($R0 = 2.5$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

## With vaccination blocking Symptoms

In [ ]:
print("******************** Running vaccine simulation")
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]

icu_atten = pfizer_atten
r_atten = [1.0, 1.0, 1.0]

free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_symptoms_{max_dose_thresold}.csv"
    figure_file = f""
    run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, 
        max_doses, doses_min_window, doses_max_window)

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.8*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Relaxed days ($R0 > 2.0$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

In [ ]:
free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_symptoms_{max_dose_thresold}.csv"
    figure_file = f""

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] >= basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Fully open ($R0 = 2.5$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

### Oxford-Astra Zeneca - block infection

In [ ]:
print("******************** Running vaccine simulation")
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]

r_atten = oxford_atten
icu_atten = [1.0, 1.0, 1.0]

free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_infection_{max_dose_thresold}_oxford.csv"
    figure_file = f""
    run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, 
        max_doses, doses_min_window, doses_max_window)

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.8*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Relaxed days ($R0 > 2.0$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

In [ ]:
free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_infection_{max_dose_thresold}_oxford.csv"
    figure_file = f""

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.95*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Fully open ($R0 > 2.375$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

In [ ]:
### Oxford-Astra Zeneca - block symptoms

In [ ]:
print("******************** Running vaccine simulation")
bins = [[0, 19], [20, 49], [50, 64], [65, 200]]

icu_atten = oxford_atten
r_atten = [1.0, 1.0, 1.0]

free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_symptoms_{max_dose_thresold}_oxford.csv"
    figure_file = f""
    run_a_test(basic_prm, result_file, figure_file, cities_data, M, target, force_dif, r0pop, r_atten, icu_atten, 
        max_doses, doses_min_window, doses_max_window)

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.8*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Relaxed days ($R0 > 2.0$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()

In [ ]:
free_days = []
for max_dose_thresold in range(28, 28*3 + 1, 7):
    doses_max_window = [max_dose_thresold]
    result_file = f"results/dose_window_sensitivity_symptoms_{max_dose_thresold}_oxford.csv"
    figure_file = f""

    solution = pd.read_csv(result_file, index_col=[0, 1, 2])
    free_days.append((max_dose_thresold, (solution.loc["rt", -1, -1] > 0.95*basic_prm["rep"]).sum()))
    print(free_days)

In [ ]:
free_days = np.array(free_days)
plt.plot(free_days[:, 0], free_days[:, 1], marker="o")
plt.title("Fully open ($R0 > 2.375$)")
plt.xlabel("Max delay (in days, starting from 28)")
plt.ylabel("Free days")
plt.xticks(free_days[:, 0])
plt.yticks(free_days[:, 1])
plt.show()